<a href="https://colab.research.google.com/github/RojaCM/DEEP-LEARNING-/blob/main/BM25%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [16]:
from google.colab import files
uploaded=files.upload()

Saving data_copy.csv to data_copy.csv


In [20]:
df=pd.read_csv('data_copy.csv')

In [21]:
df1=df[:1000]

In [22]:
df1.head()

,Unnamed: 0,url,title,desc,text
0,0,https://www.contractsfinder.service.gov.uk/Published/Notice/releases/5f3d2831-0854-49c9-a1b4-db6e181fe1e5.json,"Provision of a Fixed Mechanical Equipment, Repairs &amp; Maintenance Service","Modification Notice\r\n Original description: Provision of a Fixed Mechanical Equipment, Repairs and Maintenance Service\r\n Description of the modifications\r\n Fixed mechanical equipment repairs and maintenance - short extension to contract which will allow a full review of our specification prior to going back out to the market later in the year/beginning of next year\r\n Description of the circumstances which rendered the modification necessary and explanation of the unforeseen nature of these circumstances:\r\n Members of the procurement team have been responsible for purchasing PPE and recovery plan materials to allow area offices and depots to re-open and continue to ensure any repairs and maintenance to any group properties can be completed in a safe working manner to both the tenant and member of staff.","Provision of a Fixed Mechanical Equipment, Repairs &amp; Maintenance Service Modification Notice\r\n Original description: Provision of a Fixed Mechanical Equipment, Repairs and Maintenance Service\r\n Description of the modifications\r\n Fixed mechanical equipment repairs and maintenance - short extension to contract which will allow a full review of our specification prior to going back out to the market later in the year/beginning of next year\r\n Description of the circumstances which rendered the modification necessary and explanation of the unforeseen nature of these circumstances:\r\n Members of the procurement team have been responsible for purchasing PPE and recovery plan materials to allow area offices and depots to re-open and continue to ensure any repairs and maintenance to any group properties can be completed in a safe working manner to both the tenant and member of staff."
1,1,https://www.contractsfinder.service.gov.uk/Published/Notice/releases/6e4cfac6-cb44-495a-a58e-c57b2e14b842.json,Tender for LIft Maintenance and Associated Works,Modification Notice\r\n Original description: Provide passenger lift maintenance and associated works for a 5 year period\r\n Description of the modifications\r\n Lift maintenance and associated works - short extension to contract which will also allow a full review of our specification prior to going back out to the market later in the year.\r\n Description of the circumstances which rendered the modification necessary and explanation of the unforeseen nature of these circumstances:\r\n Members of the procurement team have been responsible for purchasing PPE and recovery plan materials to allow area offices and depots to re-open and continue to ensure any repairs and maintenance to any group properties can be completed in a safe working manner to both the tenant and member of staff,Tender for LIft Maintenance and Associated Works Modification Notice\r\n Original description: Provide passenger lift maintenance and associated works for a 5 year period\r\n Description of the modifications\r\n Lift maintenance and associated works - short extension to contract which will also allow a full review of our specification prior to going back out to the market later in the year.\r\n Description of the circumstances which rendered the modification necessary and explanation of the unforeseen nature of these circumstances:\r\n Members of the procurement team have been responsible for purchasing PPE and recovery plan materials to allow area offices and depots to re-open and continue to ensure any repairs and maintenance to any group properties can be completed in a safe working manner to both the tenant and member of staff
2,2,https://www.contractsfinder.service.gov.uk/Published/Notice/releases/51934652-4f81-4a43-82d2-4ac4b4dd76f2.json,Deployed Force Radiation Detection Monitoring Equipment Replacement (DFRDME),The Chemical Biological Radiological and Nuclear Pro

In [23]:
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import spacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")
tok_text=[] # for our tokenised corpus
#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(df1.text.str.lower().values, disable=["tagger", "parser","ner"])):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

1000it [00:06, 162.01it/s]


In [25]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [36]:
from nltk.corpus import stopwords
from textblob import Word
stop = stopwords.words('english')
df1['text1'] = df1['text'].apply(lambda x:' '.join(x.lower() for x in x.split()))
df1['text1']= df1['text'].str.replace('[^\w\s]','')
df1['text1']= df1['text'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
df1['text1'] = df1['text'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
df1['text1'] = df1['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning

In [38]:
from gensim.models import Word2Vec
Bigger_list=[]
for i in df1['text1']:
     li = list(i.split(" "))
     Bigger_list.append(li)	
Model= Word2Vec(Bigger_list,min_count=1,size=300,workers=4)

In [39]:
list(Model.wv.vocab)

['Provision',
 'of',
 'a',
 'Fixed',
 'Mechanical',
 'Equipment,',
 'Repairs',
 '&amp;',
 'Maintenance',
 'Service',
 'Modification',
 'Notice',
 'Original',
 'description:',
 'and',
 'Description',
 'the',
 'modification',
 'mechanical',
 'equipment',
 'repair',
 'maintenance',
 '-',
 'short',
 'extension',
 'to',
 'contract',
 'which',
 'will',
 'allow',
 'full',
 'review',
 'our',
 'specification',
 'prior',
 'going',
 'back',
 'out',
 'market',
 'later',
 'in',
 'year/beginning',
 'next',
 'year',
 'circumstance',
 'rendered',
 'necessary',
 'explanation',
 'unforeseen',
 'nature',
 'these',
 'circumstances:',
 'Members',
 'procurement',
 'team',
 'have',
 'been',
 'responsible',
 'for',
 'purchasing',
 'PPE',
 'recovery',
 'plan',
 'material',
 'area',
 'office',
 'depot',
 're-open',
 'continue',
 'ensure',
 'any',
 'group',
 'property',
 'can',
 'be',
 'completed',
 'safe',
 'working',
 'manner',
 'both',
 'tenant',
 'member',
 'staff.',
 'Tender',
 'LIft',
 'Associated',
 'Work

In [40]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(list(Model.wv.vocab),k1=2,b=0.8)

In [42]:
query = "Medical"
tokenized_query = query.lower().split(" ")
import time

t0 = time.time()
results = bm25.get_top_n(tokenized_query, df1.text1.values, n=10)
t1 = time.time()
print(f'Searched 50,000 records in {round(t1-t0,3) } seconds \n')
for i in results:
  print(i)

AssertionError: ignored

WORD EMBEDDING

In [1]:
import gensim

In [2]:
from nltk.corpus import abc

In [8]:
model=gensim.models.Word2Vec(abc.sents())

In [7]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /root/nltk_data...
[nltk_data]   Package abc is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
x=list(model.wv.vocab)

In [10]:
x

['PM',
 'denies',
 'knowledge',
 'of',
 'AWB',
 'kickbacks',
 'The',
 'Prime',
 'Minister',
 'has',
 'denied',
 'he',
 'knew',
 'was',
 'paying',
 'to',
 'Iraq',
 'despite',
 'writing',
 'the',
 'wheat',
 'exporter',
 'asking',
 'be',
 'kept',
 'fully',
 'informed',
 'on',
 'sales',
 '.',
 'Letters',
 'from',
 'John',
 'Howard',
 'and',
 'Deputy',
 'Mark',
 'Vaile',
 'have',
 'been',
 'released',
 'by',
 'Cole',
 'inquiry',
 'into',
 'oil',
 'for',
 'food',
 'program',
 'In',
 'one',
 'letters',
 'Mr',
 'asks',
 'managing',
 'director',
 'Andrew',
 'Lindberg',
 'remain',
 'in',
 'close',
 'contact',
 'with',
 'Government',
 'Opposition',
 "'",
 's',
 'Gavan',
 'O',
 'Connor',
 'says',
 'letter',
 'sent',
 '2002',
 ',',
 'same',
 'time',
 'though',
 'a',
 'trucking',
 'company',
 'He',
 'can',
 'longer',
 'wipe',
 'its',
 'hands',
 'illicit',
 'payments',
 'which',
 '$',
 '290',
 'million',
 '"',
 'responsibility',
 'this',
 'must',
 'lay',
 'may',
 'at',
 'feet',
 'Coalition',
 'minist

In [12]:
data=model.most_similar('money')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [13]:
data

[('information', 0.9257799386978149),
 ('put', 0.9119118452072144),
 ('difficult', 0.906688928604126),
 ('us', 0.8937133550643921),
 ('move', 0.891079306602478),
 ('themselves', 0.8860955238342285),
 ('drive', 0.8827561140060425),
 ('easy', 0.8802821040153503),
 ('effective', 0.8795716166496277),
 ('making', 0.878446638584137)]

In [17]:
model.wv.similarity('please','see')

KeyError: ignored